# 市场关注

- 作者：DGC'Idea
- 版本：V0.1  
- 更新日期：2018年4月12日

In [1]:


# 导入基本库
import numpy as np
import pandas as pd
import json
from six import StringIO
from datetime import timedelta,date

# 导入自定义库
import sys
sys.path.append('/home/jquser')
from ds_sina import *
from ds_wall import *
from ds_jisilu import *
#研究、策略中区别配置
try:
    #策略中必须导入kuanke.user_space_api包，用于支持read_file
    from kuanke.user_space_api import read_file,get_price,normalize_code
except:
    pass
    

FILE_PATH='Data/'


    
#\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#日报：市场关注（微信）
#-------------------------------------------------------------------------------------------------------------------------------------------------
        
# 基金溢价
def get_discount_fund2(value=3.0,type_list={'index_etf':'ETF','index_lof':'LOF','t0_qdii':'QDII'}):
    #标题
    info='\n基金\n'
    info1='溢价'
    info2='折价'
    for code,name in type_list.items():
        #爬取集思录数据
        df=get_jsl_data(code)
        #去除溢价率中的百分号
        df['discount_rt']=df['discount_rt'].str.replace('%','')
        #转换为浮点数字
        df['discount_rt']=df['discount_rt'].astype('float')
        #筛选所有符合标准的溢价和折价
        info1=info1+' %s%s'%(name,str(len(df[df['discount_rt']>=value])).rjust(2))
        info2=info2+' %s%s'%(name,str(len(df[df['discount_rt']<=-value])).rjust(2))
    return info+info1+'\n'+info2+'\n'


# 套利
def get_discount_fund():
    #爬取集思录数据
    df=get_jsl_data('t0_qdii')
    df=df[df['fund_id']=='162411']
    print df
    info=pd.DataFrame(
            data=[
                ['#套利','','','','',''],
                ['华宝油气','溢价率','当前价格',
                 '%s日净值'%(df['nav_dt'].iloc[0][-2:]),'%s日估值'%(df['est_val_dt'].iloc[0][-2:]),
                 '指数涨幅'],
                ['',df['discount_rt'].iloc[0],df['price'].iloc[0],
                 round(float(df['fund_nav'].iloc[0]),3),round(float(df['estimate_value'].iloc[0]),3),
                df['ref_increase_rt'].iloc[0]]],
            columns=range(6)) 
    return info



# 新股
def get_new_stock():
    #爬取集思录数据
    a_df=get_jsl_data('new_stock')
    #获取今日日期并转换格式
    today=pd.datetime.today().strftime('%m-%d')
    #筛选A股今日新股（生成股票名称列表）
    #爬取集思录数据
    h_df=get_jsl_data('hnew_stock')
    #筛选港股今日新股（生成股票名称列表）
    info=pd.DataFrame(
                  data=[['#新股','','','','',''],
                        ['A股']+a_df[a_df['apply_dt'].str[0:5]==today]['stock_nm'][0:4].tolist(),
                        ['港股']+h_df[h_df['apply_dt'].str[0:5]==today]['stock_nm'][0:4].tolist()],
                  columns=range(6)) 
    #返回信息
    return info


#可转债
def get_new_cbond():
    #爬取集思录数据
    df=get_jsl_data('new_cbond')
    #获取今日日期并转换格式
    today=pd.datetime.today().strftime('%m-%d')
    #筛选今日可申购转债（ap_flag为A）
    #筛选今日上市转债（ap_flag为D）
    info=pd.DataFrame(
        data=[['#可转债','','','','',''],
            ['申购']+df[df['ap_flag'].str[0:1]=='A']['bond_nm'].tolist(),
            ['上市']+df[df['ap_flag'].str[0:1]=='D']['bond_nm'].tolist()],
        columns=range(6)) 
    #返回信息
    return info


#逆回购
def get_rep_money():
    #爬取集思录数据（上海逆回购）
    hs_df=get_jsl_data('sh_money')[0:5] 
    
    #爬取集思录数据（深圳逆回购）
    ss_df=get_jsl_data('sz_money')[0:5] 

    info=pd.DataFrame(
        data=[
            ['#逆回购','','','','',''],
            ['沪市']+['%s/%s'%(price,d) for d,price in zip([1,2,3,4,7],hs_df['price'].tolist())],
            ['深市']+['%s/%s'%(price,d) for d,price in zip([1,2,3,4,7],ss_df['price'].tolist())]
            ],
        columns=range(6))  
    #返回信息
    return info


#货基
def get_t0_money():
    #待选货基列表
    fund_list=['511850','511800','511830','511700','511820','511900',
        '511690','511810','511660','511990','511880']
    #爬取集思录数据
    df=get_jsl_data('t0_money')[['fund_id','sell_price']] 
    #按照待选列表筛选数据
    df=df[df['fund_id'].isin(fund_list)]
    #转换价格为浮点类型，否则排序不准确
    df['sell_price']=np.round(df['sell_price'].astype('float'),4)
    #从高到底排序,取价格最低的前4个
    df=df.sort('sell_price',ascending=True)[0:6]

    info=pd.DataFrame(
        data=[
            ['#货基','','','','',''],
            df['fund_id'].tolist(),df['sell_price'].tolist()],
        columns=range(6))  

    #返回信息
    return info


#指数估值      
def get_value_index():
    #指数关注列表       
    index_list=[
        'HSCEI',#恒生国企,
        'HSI',#恒生指数
        'SPX',#恒生指数
        '000902.SH',#全市场
        '000300.SH',#沪深300
        '000905.SH',#中证500
        '399006.sz',#创业板指
        '399005.sz',#中小板指  
        '000991.SH',#全指医药
        '000992.SH',#全指金融
        '000990.SH',#全指消费
        '000993.SH',#全指信息
        '399812.sz',#中证养老
        '000807.SH',#食品饮料
        '000922.SH',#中证红利 
        '399967.sz',#中证军工
        '399989.sz',#中证医疗
        '000827.SH',#中证环保
        '399971.sz',#中证传媒
        '399986.sz',#中证银行
        '399975.sz',#全指证券
        '000015.SH',#上证红利
        ]
    #读取指数分析表
    df=pd.read_csv(StringIO(read_file(FILE_PATH+'idx_value.csv')),index_col=0)
    df=df[df['aid']==10]
    #按照关注列表筛选
    df=df[df.index.isin(index_list)]
    #全市场估值
    code='000902.SH'
    #当前估值、百分位、区间
    #区间为极低的前5个指数名称
    info=pd.DataFrame(
        data=[
            ['#估值','','','','',''],
            ['全市场','PE:%s'%(df.ix[code,'pe_e']),'%s%%%s'%(df.ix[code,'pe_e_ratio'],df.ix[code,'pe_e_state']),
                      'PB:%s'%(df.ix[code,'pb_e']),'%s%%%s'%(df.ix[code,'pb_e_ratio'],df.ix[code,'pb_e_state'])],
            ['关注']+df[df['pe_e_state']=='极低']['name'][0:5].tolist() ],
        columns=range(6))  
    #返回信息
    return info
    

    
#实时价格    
def get_real_price():
    #获取wall实时价格
    price=Wall.real_price('000001.SS,399300.SZ,399006.SZ,399001.SZ,\
SPX500INDEX,US30INDEX,NASINDEX,HKG33INDEX,GER30INDEX,SP500VIXINDEX,UK100INDEX,BDIINDEX,\
USDOLLARINDEX,USDCNH,USDCNY,USDJPY,HKDCNY,BTCUSD,\
XAUUSD,XAGUSD,\
UKOIL,USOIL,\
US10YEAR,US5YEAR,US3YEAR,CHINA10YEAR,CHINA5YEAR,CHINA3YEAR')
    info=pd.DataFrame(
        data=[
            ['#股指','','','','',''],
            ['上证综指','沪深300','创业板指','深证成指','恒生指数','BDI指数'],
            [price['000001.SS'],price['399300.SZ'],price['399006.SZ'],price['399001.SZ'],price['HKG33INDEX'],price['BDIINDEX']],
            ['标普500','道琼斯','纳斯达克','德国DAX','英国富时','VIX波动'],
            [price['SPX500INDEX'],price['US30INDEX'],price['NASINDEX'],price['GER30INDEX'],
             price['UK100INDEX'],price['SP500VIXINDEX']],
            ['#商品','','','','',''],
            ['黄金','白银','金银比','布原油','WTI原油','金油比'],
            [price['XAUUSD'],price['XAGUSD'],round(price['XAUUSD']/price['XAGUSD'],3),
            price['UKOIL'],price['USOIL'],round(price['XAUUSD']/price['UKOIL'],3)],
            ['#外汇','','','','',''],
            ['美元指数','离岸￥','在岸￥','港币/￥','＄/日元 ','比特币/＄'],
            [price['USDOLLARINDEX'],price['USDCNH'],price['USDCNY'],price['HKDCNY'],price['USDJPY'],price['BTCUSD']],
            ['#国债','','','','',''],
            ['美国10年','美国5年','美国3年','中国10年','中国5年','中国3年'],
            [price['USDOLLARINDEX'],price['USDJPY'],price['USDCNH'],
            price['CHINA10YEAR'],price['CHINA5YEAR'],price['CHINA3YEAR']]
            ],
          columns=range(6)) 

    #返回信息
    return info       
    
    
#获取关注信息     
def get_market_watch():

    df=pd.concat([get_value_index(),get_real_price(),get_discount_fund(),get_rep_money(),
                  get_t0_money(),get_new_stock(),get_new_cbond()])
    return df.to_html(
            index=False,
            header=False,
            classes='')\
            .replace('<table border="1" class="dataframe">','<table border="1" cellspacing="0" cellpadding="6" rules="rows" frame="void">')\
            .replace('<td>#','<td style="text-align:left;font-weight:bold">')\
            .replace('<tbody>','<tbody style="text-align:right;font-size:8px">')\
            .replace('None','')

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)

#get_discount_fund()
print get_market_watch()    
# print get_market_watch()
# print get_t0_money()

   amount amount_incr        amount_incr_tips amount_increase_rt apply_fee                                     apply_fee_tips apply_redeem_status apply_status asset_ratio discount_rt  est_val_dt est_val_increase_rt estimate_value fund_id fund_id_color fund_nav fund_nm fund_nm_color increase_rt index_id       index_nm    last_est_datetime last_est_dt last_est_time last_time lof_type min_amt money_cd      nav_dt notes  owned  price    price_dt redeem_fee                                  redeem_fee_tips redeem_status ref_increase_rt stock_volume turnover_rt   volume
9  227363         788  最新份额：227363万份；增长：0.35%               0.35      1.5%  50万以下 1.5%；\n50万(含)至100万 1.2%；\n100万(含)至200万 1...           开放申购/开放赎回         开放申购      95.000      -1.23%  2019-03-18               2.25%         0.5123  162411        162411   0.5010    华宝油气          华宝油气       1.20%        Y  标普石油天然气上游股票指数  2019-03-19 15:00:03  2019-03-19      15:00:03  15:00:03     QDII    None      USD  2019-03-15            0  0.

<table border="1" class="dataframe border="1" cellspacing="0" cellpadding="6" rules="rows" frame="void"">
  <thead>
    <tr style="text-align: right;">
      <th>0</th>
      <th>1</th>
      <th>2</th>
      <th>3</th>
      <th>4</th>
      <th>5</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="left">逆回购</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td>204001</td>
      <td>204002</td>
      <td>204003</td>
      <td>204004</td>
      <td>204007</td>
      <td>204014</td>
    </tr>
    <tr>
      <td>2.340</td>
      <td>2.300</td>
      <td>2.390</td>
      <td>2.310</td>
      <td>2.435</td>
      <td>2.505</td>
    </tr>
    <tr>
      <td>131810</td>
      <td>131811</td>
      <td>131800</td>
      <td>131809</td>
      <td>131801</td>
      <td>131802</td>
    </tr>
    <tr>
      <td>1.701</td>
      <td>1.600</td>
      <td>1.800</td>
      <td>1.900</td>
      <td>2.250</td>
      <td>2.302</td>
    </tr>
    <tr>
      <td align="left">货基</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td>511820</td>
      <td>511810</td>
      <td>511900</td>
      <td>511800</td>
      <td>None</td>
      <td>None</td>
    </tr>
    <tr>
      <td>100.0024</td>
      <td>100.0034</td>
      <td>100.0045</td>
      <td>100.0057</td>
      <td>None</td>
      <td>None</td>
    </tr>
  </tbody>
</table>

In [1]:
import sys
sys.path.append('/home/jquser')
from at_watch import *

In [2]:
print get_market_watch()

<table border="1" class="dataframe border="1" cellspacing="0" cellpadding="6" rules="rows" frame="void"">
  <tbody style="text-align:right;">
    <tr>
      <td style="text-align:left;font-weight:bold">指数估值</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td>全市场</td>
      <td>PE:24.66</td>
      <td>3.62%极低</td>
      <td>PB:1.85</td>
      <td>2.63%极低</td>
      <td></td>
    </tr>
    <tr>
      <td>关注</td>
      <td>全指消费</td>
      <td>沪深300</td>
      <td>中证500</td>
      <td>全指信息</td>
      <td>中证红利</td>
    </tr>
    <tr>
      <td style="text-align:left;font-weight:bold">股指</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td>上证综指</td>
      <td>沪深300</td>
      <td>创业板指</td>
      <td>深证成指</td>
      <td>恒生指数</td>
      <td></td>
    </tr>
    <tr>
      <td>2692.369</td>
      <td>3353.381</td>
      <td>1363.527</td>
      <td>8160.279</td>
      <td>279

## 1.简讯

In [1]:
#导入函数库
import numpy as np
import pandas as pd
import json
from six import StringIO
from datetime import timedelta,date

#聚源数据库
from jqdata import jy

import os

#引入自定义（jsl）库
#保存当前目录
path=os.getcwd()
#更改到用户根目录为
os.chdir("/home/jquser/")
#导入自己的库
from jsl import *
from sina import *
from mail import *
from index_base import *
#恢复当前目录
os.chdir(path)
#导入自定义库（从集思录爬取数据）


#读取指数估值分析表        
def read_analysis_index(file_root='../指数/'):
    #文件名
    file_name='%s%s.csv'%(file_root,'index_analysis') 
    #从cvs文件恢复数据
    df=pd.read_csv(file_name,index_col=0,parse_dates=False)
    return df

#基金溢价
def get_discount_fund(value=3.0,type_list={'index_etf':'ETF','index_lof':'LOF','t0_qdii':'QDII'}):
    #标题
    info='基金'
    info1='溢价'
    info2='折价'
    for code,name in type_list.items():
        #爬取集思录数据
        df=get_jsl_data(code)
        #去除溢价率中的百分号
        df['discount_rt']=df['discount_rt'].str.replace('%','')
        #转换为浮点数字
        df['discount_rt']=df['discount_rt'].astype('float')
        #筛选所有符合标准的溢价和折价
        info1=info1+' %s%s'%(name,str(len(df[df['discount_rt']>=value])).rjust(2))
        info2=info2+' %s%s'%(name,str(len(df[df['discount_rt']<=-value])).rjust(2))
    return info+'\n'+info1+'\n'+info2+'\n'

#新股
def get_new_stock():
    #标题
    info='新股\n'  
    #爬取集思录数据
    df=get_jsl_data('new_stock')
    #获取今日日期并转换格式
    today=pd.datetime.today().strftime('%m-%d')
    #筛选A股今日新股（生成股票名称列表）
    info=info+'A股 %s\n'%(trans_list(df[df['apply_dt'].str[0:5]==today]['stock_nm'].tolist()))
    #爬取集思录数据
    df=get_jsl_data('hnew_stock')
    #筛选港股今日新股（生成股票名称列表）
    info=info+'港股 %s\n'%(trans_list(df[df['apply_dt'].str[0:5]==today]['stock_nm'].tolist()))
    #返回信息
    return info

#可转债
def get_new_cbond():
    #标题
    info='可转债\n'  
    #爬取集思录数据
    df=get_jsl_data('new_cbond')
    #获取今日日期并转换格式
    today=pd.datetime.today().strftime('%m-%d')
    #筛选今日可申购转债
    info=info+'申购 %s\n'%(trans_list(df[df['ap_flag'].str[0:1]=='A']['apply_cd'].tolist()))
    #筛选今日上市转债
    info=info+'上市 %s\n'%(trans_list(df[df['ap_flag'].str[0:1]=='D']['bond_nm'].tolist()).replace('转债',''))
    #返回信息
    return info

#逆回购
def get_rep_money():
    #标题
    info='逆回购\n' 
    #天数标题
    #info=info+'天 1     2     3     4     5     6     7     \n'
    #爬取集思录数据
    df=get_jsl_data('sh_money') 
    df['price']=df['price'].str[:-2].str.rjust(4)
    #沪市逆回购价格聊表（取7天内）
    info=info+'沪 %s\n'%(trans_list(df['price'].tolist()[0:7]))
    #爬取集思录数据
    df=get_jsl_data('sz_money') 
    df['price']=df['price'].str[:-2].str.rjust(4)
    #深市逆回购价格列表（取7天内）
    info=info+'深 %s\n'%(trans_list(df['price'].tolist()[0:7]))
    #返回信息
    return info

#货基
def get_t0_money():
    #标题
    info='T+0货基\n' 
    #待选货基列表
    fund_list=['511800','511830','511700','511820','511900',
        '511690','511810','511660','511850','511990','511880']
    #爬取集思录数据
    df=get_jsl_data('t0_money')[['fund_id','sell_price']] 
    #按照待选列表筛选数据
    df=df[df['fund_id'].isin(fund_list)]
    #转换价格格式及排序
    df['sell_price']=df['sell_price'].astype('float32')
    df=df.sort('sell_price',ascending=True)
    #取价格最低的前4个
    df=df[0:4]
    #去除多余的小数位，按照7位补足前面空格
    df['sell_price']=df['sell_price'].astype('str').str[:-2].str.rjust(6)
    #代码
    info=info+'代码 %s\n'%(trans_list(df['fund_id'].tolist()))
    #价格
    info=info+'价格 %s\n'%(trans_list(df['sell_price'].tolist()))
    #返回信息
    return info

#指数估值      
def get_value_index():
    #标题
    info='指数估值\n'
    df=read_index_analysis()
    df=df[df.index.isin(index_use_list)]
    #全市场估值
    code='000902.XSHG'
    info=info+'全市场 %s %s%% %s\n'%(df.ix[code,'pe'],df.ix[code,'pe_ratio'],df.ix[code,'pe_state'])
    #低估的
    info=info+'关注　 %s\n'%(trans_list(df[df['pe_state']=='极低']['index'][0:3].tolist()))
    #返回信息
    return info
    
#商品价格
def get_price_commodity():
    #标题
    info='商品\n' 
    #爬取新浪实时数据
    data=get_sina_hf_price(['hf_GC','hf_SI','hf_OIL'])  
    #沪市逆回购价格聊表（取7天内）
    info=info+'金%s 银%s 油%s 金银比%.2f\n'%(data['hf_GC'],
        data['hf_SI'],data['hf_OIL'],float(data['hf_GC'])/float(data['hf_SI']))
    #返回信息
    return info   
    
#去除列表中的多余字符
def trans_list(lst):
    #解决中文乱码问题
    data=json.dumps(lst,encoding='UTF-8',ensure_ascii=False)
    #去除列表中的多余字符
    data=data.replace('"','').replace(',','').replace('[','').replace(']','')
    #返回字符串，如果是无内容则为'-'
    return (data if data!='' else '-')
    
#获取关注信息     
def get_market_notice():
    (info_1,info_2,info_3,info_4,info_5,info_6,info_7)='','','','','','',''
    try:
        info_1=get_value_index()
        info_2=get_new_cbond()
        info_3=get_new_stock()
        info_4=get_price_commodity()
        info_5=get_rep_money()
        info_6=get_discount_fund()
        info_7=get_t0_money()
    except:
        raise 
        #pass
    notice='日报：市场关注\n\n%s\n%s\n%s\n%s\n%s\n%s\n%s'%(
        info_1,
        info_2,
        info_3,
        info_4,
        info_5,
        info_6,
        info_7,
        )
    return notice#.replace('\n','<br>')

notice=get_market_notice()
print notice

ImportError: No module named jsl

In [1]:
import pandas as pd 
import numpy as np

import sys
sys.path.append('/home/jquser')
from wall import *




In [2]:
def real_price(code):  
    """
    获取wall网站财经数据实时价格
    
    code：代码，str，多个代码用','分隔
    返回：代码和最新价格的字典，dict
    
    返回数据实例
    数据在data、snapshot中
    fields为字段名
    
    A股指数返回数据：
    无en_name字段
    {u'code': 200, u'data': {u'snapshot': {u'fields': [u'prod_name', u'last_px', u'px_change', u'px_change_rate', 
    u'high_px', u'low_px', u'open_px', u'preclose_px', u'business_amount', u'trade_status', u'securities_type', 
    u'update_time', u'price_precision', u'en_name'], u'000001.SS': [u'\u4e0a\u8bc1\u6307\u6570', 2786.8966, 
    -26.280899999999747, -0.9342069599234228, 2825.9872, 2782.4464, 2799.9039, 2813.1775, 11835790700, u'BREAK', 
    u'index', 1530169200, 2, u'000001.SS']}}}
    
    海外指数、商品、外汇等返回数据：
    有en_name字段
    {u'code': 200, u'data': {u'snapshot': {u'fields': [u'en_name', u'prod_name', u'last_px', u'px_change', 
    u'px_change_rate', u'high_px', u'low_px', u'open_px', u'preclose_px', u'business_amount', u'trade_status',
    u'securities_type', u'update_time', u'price_precision', u'week_52_high', u'week_52_low'], u'SPX500INDEX': 
    [u'SPX', u'\u6807\u666e500', 2699.63, -23.43, -0.86, 2746.09, 2699.38, 2723.06, 2723.06, 0, u'ENDTR', u'indice',
    1530129598, 2, 2872.87, 2406.55]}}}
    """
    
    # url，换行前面不能有空格和tab
    url='https://forexdata.wallstreetcn.com/real?en_prod_code={en_prod_code}&\
fields=en_name,prod_name,last_px,px_change,px_change_rate,high_px,low_px,open_px,preclose_px,\
business_amount,business_balance,market_value,turnover_ratio,dyn_pb_rate,amplitude,pe_rate,bps,\
hq_type_code,trade_status,bid_grp,offer_grp,business_amount_in,business_amount_out,circulation_value,\
securities_type,update_time,price_precision,week_52_high,week_52_low'.format(en_prod_code=code)
    # 请求数据
    response=requests.get(url)
    # json解析,有效数据在snapshot中
    json_data=json.loads(response.text).get("data").get("snapshot")
    # 删除fields
    json_data.pop('fields')
    # 判断code中
    dot_count=code.strip(',').count('.')
    if dot_count==0 : 
        n=2
    elif dot_count>0 : 
        n=1
    data={}
    for item in json_data:
        data[item]=float(json_data.get(item)[n])
    return data


# print real_price('SPX500INDEX,399300.SZ')



def get_real_price():
    #获取wall实时价格
    price=Wall.real_price('000001.SS,399300.SZ,399006.SZ,\
SPX500INDEX,US30INDEX,NASINDEX,HKG33INDEX,GER30INDEX,SP500VIXINDEX,\
USDOLLARINDEX,USDCNH,USDJPY,\
XAUUSD,XAGUSD,\
UKOIL,USOIL,\
US10YEAR,US5YEAR,US3YEAR,CHINA10YEAR,CHINA5YEAR,CHINA3YEAR')
    info='\n股指\n' 
    info+='%s %s %s\n'%('上证综指　','沪深300　 ','创业板指')
    info+='%-10.2f %-10.2f %-10.2f\n'%(price['000001.SS'],price['399300.SZ'],price['399006.SZ'])
    info+='%s %s %s\n'%('标普500　 ','道琼斯　　','纳斯达克')
    info+='%-10.2f %-10.2f %-10.2f\n'%(price['SPX500INDEX'],price['US30INDEX'],price['NASINDEX'])
    info+='%s %s %s\n'%('恒生指数　','德国DAX　 ','VIX波动')
    info+='%-10.2f %-10.2f %-10.2f\n'%(price['HKG33INDEX'],price['GER30INDEX'],price['SP500VIXINDEX'])
    info+='\n商品\n' 
    info+='%s %s %s\n'%('黄金　　　','白银　　　','金银比')
    info+='%-10.2f %-10.2f %-10.2f\n'%(price['XAUUSD'],price['XAGUSD'],price['XAUUSD']/price['XAGUSD'])    
    info+='%s %s %s\n'%('布原油　　','WTI原油　 ','金油比')
    info+='%-10.2f %-10.2f %-10.2f\n'%(price['UKOIL'],price['USOIL'],price['XAUUSD']/price['UKOIL'])  
    info+='\n外汇\n' 
    info+='%s %s %s\n'%('美元指数　','美元/日元 ','离岸人民币')
    info+='%-10.2f %-10.2f %-10.2f\n'%(price['USDOLLARINDEX'],price['USDJPY'],price['USDCNH'])  
    info+='\n国债\n' 
    info+='%s %s %s\n'%('美国10年　','美国5年　 ','美国3年')
    info+='%-10.2f %-10.2f %-10.2f\n'%(price['US10YEAR'],price['US5YEAR'],price['US3YEAR'])
    info+='%s %s %s\n'%('中国10年　','中国5年　 ','中国3年')
    info+='%-10.2f %-10.2f %-10.2f\n'%(price['CHINA10YEAR'],price['CHINA5YEAR'],price['CHINA3YEAR'])
    #返回信息
    return info   

temp=get_real_price()
print temp





    #爬取新浪实时数据
#     data=get_sina_hf_price(['hf_GC','hf_SI','hf_OIL'])  
#     #金价、银价、油价、金银比
#     info=info+'金%.2f 银%.2f 油%.2f 金银比%.2f\n'%(float(data['hf_GC']),
#         float(data['hf_SI']),float(data['hf_OIL']),float(data['hf_GC'])/float(data['hf_SI']))


股指
上证综指　 沪深300　  创业板指
2784.03    3426.19    1548.44   
标普500　  道琼斯　　 纳斯达克
2716.31    24216.05   7503.68   
恒生指数　 德国DAX　  VIX波动
28567.38   12177.23   16.85     

商品
黄金　　　 白银　　　 金银比
1247.70    15.99      78.03     
布原油　　 WTI原油　  金油比
77.23      73.03      16.16     

外汇
美元指数　 美元/日元  离岸人民币
95.30      110.51     6.65      

国债
美国10年　 美国5年　  美国3年
2.84       2.72       2.60      
中国10年　 中国5年　  中国3年
3.57       3.44       3.39      



In [4]:
'333335'[:-1]

'33333'